In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
path_original = '/content/drive/MyDrive/Colab Notebooks/mafs_cBioPortal/Original/'
path_clean = '/content/drive/MyDrive/Colab Notebooks/mafs_cBioPortal/Clean/'

In [ ]:
def read_maf(maf_file_name):
    maf_fields = ["Hugo_Symbol", "Chromosome", "Start_Position", "End_Position", "Reference_Allele", "Tumor_Seq_Allele2", "Variant_Classification", "Variant_Type", "Tumor_Sample_Barcode"]
    maf = pd.read_csv(maf_file_name, sep="\t", usecols=maf_fields,comment="#")
    return maf

In [ ]:
def get_mutations_per_patient(maf):
    mutations_per_patient = {}
    patients = set(maf["Tumor_Sample_Barcode"])
    for patient in patients:
        maf_one_gene = maf[maf.Tumor_Sample_Barcode.isin([patient])]
        mutations_per_patient[patient] = len(maf_one_gene)
    return mutations_per_patient

In [ ]:
def remove_hipermutated_patients(maf, mutations_per_patient):
    mutations_per_patient_count = list(mutations_per_patient.values())
    q1 = np.quantile(mutations_per_patient_count, .25)
    q2 = np.quantile(mutations_per_patient_count, .50)
    q3 = np.quantile(mutations_per_patient_count, .75)
    iqr = q3 - q1
    threshold_hm = q3 + 4.5*iqr
    mutations_per_patient_filtered = dict(filter(lambda elem: elem[1] <= threshold_hm, mutations_per_patient.items()))
    maf = maf[maf.Tumor_Sample_Barcode.isin(list(mutations_per_patient_filtered))]
    return maf

In [ ]:
"""
Gera uma lista com o nome de todos os mafs do diretório
"""
def mafs_list_in_dir(path):
    l_mafs = []
    for _, _, file in os.walk(path):
        l_mafs = file
    return l_mafs

def convert_mafcBioPoral_mafClean(l_mafs_name):
    for maf in l_mafs_name:
        maf_df = read_maf(path_original + maf)
        # Create a new file
        new_maf_name = maf
        new_maf_name = new_maf_name.replace('.txt', '.tsv')
        # Remove hipermutated
        mafClean = remove_hipermutated_patients(maf_df, get_mutations_per_patient(maf_df))
        mafClean.to_csv(path_clean + new_maf_name,sep='\t')


In [ ]:
l_mafs_name = mafs_list_in_dir(path_original)
#print(l_mafs_name)
convert_mafcBioPoral_mafClean(l_mafs_name)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
